# Data collected from OP's website op-koti.fi. Initially I used webscraping technique to collect data. But using the API was more dynamic and easy to handle

In [1]:
import numpy as np
import pandas as pd
import requests
# import math
import re
from pandas import json_normalize

In [2]:
URL = 'https://op-koti.fi/api/apartments?mode=sale&featureGroup=apartment&orderBy=created&order=desc&offset=0&limit=4500'
# request the URL and parse the JSON
response = requests.get(URL)
response.raise_for_status() # raise exception if invalid response
data = response.json()['listings']

# normalizing the json data into a dataframe
df = json_normalize(data, max_level = 1)

In [3]:
df.head(10)

,id,kipinaId,listingType,commission,listingGroup,source,created,featureGroups,floor,year,...,siteArea.size,siteArea.displayUnit,pricing.debtFreePrice,siteArea,videoUrl,virtualPresentationUrl,livingArea,totalArea,newHousing.newHousingId,newHousing.name
0,505893,430607,90,K,82,Kipinä,2021-05-19T10:05:11.681Z,[apartment],0.0,2007.0,...,1035.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,506333,431047,89,O,82,Kipinä,2021-05-19T10:05:09.394Z,[apartment],1.0,1967.0,...,2176.9,m2,35000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,695667,420364,89,O,82,Kipinä,2021-05-19T09:05:19.419Z,[apartment],1.0,1990.0,...,1831.0,m2,32000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,682219,406917,90,K,82,Kipinä,2021-05-19T09:05:19.124Z,[apartment],NaN,2008.0,...,2587.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,689475,414178,89,O,82,Kipinä,2021-05-19T09:05:17.576Z,[apartment],2.0,1962.0,...,NaN,NaN,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,505769,430483,90,K,82,Kipinä,2021-05-19T09:05:17.258Z,[apartment],0.0,1974.0,...,5500.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,506041,430756,89,O,82,Kipinä,2021-05-19T09:05:14.076Z,[apartment],2.0,1983.0,...,5185.0,m2,169000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,506246,430960,89,O,82,Kipinä,2021-05-19T09:05:13.315Z,[apartment],4.0,1952.0,...,7349.0,m2,96000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,504280,428994,89,O,82,Kipinä,2021-05-19T09:04:59.932Z,[apartment],2.0,1940.0,...,2619.7,m2,70500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,694707,419405,90,K,82,Kipinä,2021-05-19T08:05:09.093Z,[apartment],0.0,1978.0,...,1080.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It can be seen above that the dataframe contains multiple columns that are irrelevant to us. Relevant columns are selected as shown below.

In [4]:
df.columns

Index(['id', 'kipinaId', 'listingType', 'commission', 'listingGroup', 'source',
       'created', 'featureGroups', 'floor', 'year', 'imageUrl', 'showings',
       'rooms', 'numberOfRooms', 'price', 'debtFreePrice', 'rentPerMonth',
       'location.streetAddress', 'location.postalCode', 'location.postOffice',
       'location.city', 'location.cityId', 'location.region',
       'location.regionId', 'location.district', 'location.districtId',
       'location.streetId', 'location.coordinates', 'housing.livingArea',
       'housing.totalArea', 'housing.siteArea', 'housing.rooms',
       'housing.numberOfRooms', 'pricing.price', 'livingArea.size',
       'livingArea.displayUnit', 'totalArea.size', 'totalArea.displayUnit',
       'siteArea.size', 'siteArea.displayUnit', 'pricing.debtFreePrice',
       'siteArea', 'videoUrl', 'virtualPresentationUrl', 'livingArea',
       'totalArea', 'newHousing.newHousingId', 'newHousing.name'],
      dtype='object')

In [5]:
cols = ['id', 'listingType', 'floor', 'year', 'rooms', 'numberOfRooms', 'price',
       'location.city', 'location.region', 'location.district','location.postalCode',
       'livingArea.size', 'totalArea.size']

df = df[cols]

Renaming the cols

In [6]:
df = df.rename(columns = {'location.city':'city','location.region':'region','location.district':'district','location.postalCode':'postalCode','livingArea.size':'livingArea','totalArea.size':'totalArea', 'year':'yearBuilt'})

Mapping the listingType column to the apporpriate type of house (extracted from the website manually)

In [7]:
lType = {89:'Kerrostalo',90:'Omakotitalo',91:'Rivitalo',92:'Paritalo',93:'Erillistalo',112:'Puutalo',113:'Luhtitalo',470:'Kytketty paritalo'}
df['listingType'] = df['listingType'].map(lType)

In [8]:
# some values in df.district are ''. So i decided to fill them with np.nan and then remove them altogether in the next step
df[df.district == ''] = np.nan

In [9]:
def count_NAs(df):
    for col in df:
        print(col," : ",df[col].isnull().sum())
        
def homeSizeLabels(area):
    """these ranges were calulated by using pd.cut(). 1=Smallest...4=Largest"""
    if area in range(0,100):
        return 1
    elif area in range(100,175):
        return 2
    elif area in range(175,250):
        return 3
    else:
        return 4 

In [10]:
# df = df.dropna(inplace = True)
count_NAs(df)

id  :  221
listingType  :  221
floor  :  594
yearBuilt  :  228
rooms  :  349
numberOfRooms  :  349
price  :  349
city  :  221
region  :  221
district  :  221
postalCode  :  221
livingArea  :  349
totalArea  :  349


In [11]:
# df.dropna(inplace = True)
# count_NAs(df)

Converting the columns to string and integer(from float) in order to remove the .0 at the end of figures

In [12]:
# df['id'] = df['id'].astype(str)
# df['floor'] = df['floor'].astype(int)
# df['numberOfRooms'] = df['numberOfRooms'].astype(int)
# df['yearBuilt'] = df['yearBuilt'].astype(int)
# df['postalCode'] = df['postalCode'].astype(str)

Calculating building age from 'yearBuilt'. Some buildings are supposed to be completed in the future. So any building that gets completed in the future year are also grouped into '1 year old buildings'

In [13]:
df['buildingAge'] = [1 if x > 2021 else (pd.to_datetime('today').year - pd.to_datetime(x, format = "%Y").year) for x in df.yearBuilt]
# df['centrum'] = [1 if x == 'Keskusta' else 0 for x in df.district]
df['howBig'] = [homeSizeLabels(x) for x in df.livingArea]

In [14]:
# hideRows = ['listingGroup', 'yearBuilt', 'imageUrl', 'streetAddress', 'postalCode', 'region', 'link']
df.drop('yearBuilt', axis = 1, inplace = True)
df.reset_index(drop = True)

,id,listingType,floor,rooms,numberOfRooms,price,city,region,district,postalCode,livingArea,totalArea,buildingAge,howBig
0,505893,Omakotitalo,0.0,"6 mh,oh,k,kph*2,s,lasitettu terassi,p",7.0,475000.00,Jyväskylä,Jyväskylä,Haukkala,40740,211.0,258.0,14.0,3
1,506333,Kerrostalo,1.0,"1h,kk,parv",1.0,30017.89,Kemi,Kemi,Sauvosaari,94100,21.5,21.5,54.0,4
2,695667,Kerrostalo,1.0,"1h, kk, s",1.0,31506.00,Pieksämäki,Pieksämäki,Keskusta,76100,32.5,32.5,31.0,4
3,682219,Omakotitalo,NaN,"4-5mh,tupak,tkh,khh,kph,s,3Xwc",7.0,158000.00,Nakkila,Nakkila,Lammainen,29250,119.0,198.0,13.0,2
4,689475,Kerrostalo,2.0,"2h,k,parvi",2.0,30000.00,Janakkala,Janakkala,Turenki,14200,57.5,87.5,59.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,592934,Rivitalo,1.0,"2h, k",2.0,126000.00,Rauma,Rauma,Ota,26100,81.0,81.0,85.0,1
3746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3747,578021,Paritalo,1.0,"2h,s",2.0,84446.92,Imatra,Imatra,Ukonniemi,55420,51.0,51.0,17.0,1
3748,573385,Omakotitalo,0.0,"4h,k,s+takkahuone/oleskelutila",4.0,95000.00,Sastamala,Sastamala,Keikyän Meskala,32740,119.0,119.0,47.0,2


The values in the 'rooms' column are separated using , or + or . or * and need to be cleaned

In [15]:
df.dropna(inplace = True)
count_NAs(df)

id  :  0
listingType  :  0
floor  :  0
rooms  :  0
numberOfRooms  :  0
price  :  0
city  :  0
region  :  0
district  :  0
postalCode  :  0
livingArea  :  0
totalArea  :  0
buildingAge  :  0
howBig  :  0


In [16]:
def clean_rooms(item):
    delimiters = "+", ","
    # >>> example = "stackoverflow (c) is awesome... isn't it?"
    regexPattern = '|'.join(map(re.escape, delimiters))
    return([re.sub('\\+|,|\\.|\\*| ', '', x) for x in re.split(regexPattern, item)])

df.rooms = [clean_rooms(x) for x in df.rooms]

These functions take a single entry from df.rooms and detects the features. These features are then represented with new columns using 1 or 0 denoting the presence or absence of the feature

In [17]:
# count = 0
# postals = []
# for item in df.postalCode:
#     if item[2:] == '100':
#         count+=1
#         postals.append(item)

# print(count)
# len(set(postals))

In [18]:
# df.loc[df.centrum==1]

In [19]:
def detect_centrum(postalCode):
    if postalCode[2:] == '100':
        return 1
    else:
        return 0

df['centrum'] = [detect_centrum(x) for x in df.postalCode]

def detect_sauna(room): #v4
    if 's' in room or str(room).find('sauna') != -1:
        return 1
    else:
        return 0
df['hasSauna'] = [detect_sauna(x) for x in df.rooms]

def detect_balcony(room): #v3
    if 'p' in room or str(room).find('parv') != -1:
        return 1
    else:
        return 0
df['hasBalcony'] = [detect_balcony(x) for x in df.rooms]

def detect_parking(room): #v3
    if 'ak' in room or 'at' in room or str(room).find('auto') != -1:
        return 1
    else:
        return 0
df['hasParking'] = [detect_parking(x) for x in df.rooms]

def detect_walk_in_closet(room): #v3
    if 'v' in room or 'vh' in room or str(room).find('vaate') != -1:
        return 1
    else:
        return 0
df['hasWalkInCloset'] = [detect_walk_in_closet(x) for x in df.rooms]

def detect_storage_room(room):
    if str(room).find('var') != -1:
        return 1
    else:
        return 0
df['hasStorageRoom'] = [detect_storage_room(x) for x in df.rooms]

In [20]:
# df.dropna(inplace = True)
# count_NAs(df)

In [21]:
df.drop('rooms', axis = 1, inplace = True)

In [22]:
df['id'] = df['id'].astype(str)
df['floor'] = df['floor'].astype(int)
df['numberOfRooms'] = df['numberOfRooms'].astype(int)
df['postalCode'] = df['postalCode'].astype(str)

In [23]:
df.head()

,id,listingType,floor,numberOfRooms,price,city,region,district,postalCode,livingArea,totalArea,buildingAge,howBig,centrum,hasSauna,hasBalcony,hasParking,hasWalkInCloset,hasStorageRoom
0,505893,Omakotitalo,0,7,475000.00,Jyväskylä,Jyväskylä,Haukkala,40740,211.0,258.0,14.0,3,0,1,1,0,0,0
1,506333,Kerrostalo,1,1,30017.89,Kemi,Kemi,Sauvosaari,94100,21.5,21.5,54.0,4,1,0,1,0,0,0
2,695667,Kerrostalo,1,1,31506.00,Pieksämäki,Pieksämäki,Keskusta,76100,32.5,32.5,31.0,4,1,1,0,0,0,0
4,689475,Kerrostalo,2,2,30000.00,Janakkala,Janakkala,Turenki,14200,57.5,87.5,59.0,4,0,0,1,0,0,0
5,505769,Omakotitalo,0,6,189000.00,Taivassalo,Taivassalo,Taivassalo,23310,223.2,337.8,47.0,4,0,1,0,1,0,0


In [26]:
set(df.listingType)

{'Erillistalo',
 'Kerrostalo',
 'Luhtitalo',
 'Omakotitalo',
 'Paritalo',
 'Puutalo',
 'Rivitalo'}

Exporting the dataframe to a csv file

In [25]:
df.to_csv(f'op-koti.csv', sep=',', encoding='utf-8', index=False)